In [122]:
import pandas as pd
import numpy as np
import ydata_profiling as yp
import datetime as dt
import sklearn
from tqdm.notebook import trange

In [123]:
df_train = pd.read_csv("sales_history.csv")
df_test = pd.read_csv("test_origin.csv", index_col=0)

In [124]:
df_test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


In [125]:
df_test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


In [126]:
# 予測する商品IDをテストデータから取得
items_name = df_test.iloc[:, 0].unique()

# 予測する商品IDが含まれる行を抽出
df_train = df_train[df_train.iloc[:, 2].isin(items_name)]

In [127]:
# 日付をDateTime型に変換
def ToDate(row):
    
    row.iloc[0] = dt.datetime.strptime(row.iloc[0], '%Y-%m-%d')
    row.iloc[0] = row.iloc[0].date()

    return row


df_train = df_train.apply(ToDate, axis=1)

In [128]:
# 売上個数をintに変換
df_train["売上個数"] = df_train["売上個数"].astype('int')

In [129]:
df_train

,日付,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,9,1000001,420,1
1,2018-01-01,6,1000001,420,1
2,2018-01-01,10,1000001,420,1
364,2018-01-01,0,1200007,250,1
365,2018-01-01,15,1200007,130,1
...,...,...,...,...,...
1119551,2019-10-31,0,3200026,850,1
1119553,2019-10-31,10,3300001,2040,1
1119562,2019-10-31,15,3400002,1440,1
1119563,2019-10-31,17,3400003,1190,1


In [130]:
df_train.iloc[0, 0].month

1

In [131]:
# "Month"列を追加
df_train.insert(1, "Months", 0)

t1 = dt.date(year=2018, month=1, day=1)


def AddMonths(row):

    months = abs(row[0].year - t1.year)*12 + (row[0].month - t1.month)
    row.iloc[1] = int(months)
    return row

df_train = df_train.apply(AddMonths, axis=1)


In [132]:
df_train

,日付,Months,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,0,9,1000001,420,1
1,2018-01-01,0,6,1000001,420,1
2,2018-01-01,0,10,1000001,420,1
364,2018-01-01,0,0,1200007,250,1
365,2018-01-01,0,15,1200007,130,1
...,...,...,...,...,...,...
1119551,2019-10-31,21,0,3200026,850,1
1119553,2019-10-31,21,10,3300001,2040,1
1119562,2019-10-31,21,15,3400002,1440,1
1119563,2019-10-31,21,17,3400003,1190,1


In [133]:
# 月・店舗・商品ごとに売上個数を合算

list_months_shop_id_sum = []

for m in trange(22):
    for s in trange(18):
        for i in items_name:
            list = []
            sum = df_train.loc[(df_train["Months"]==m) & (df_train["店舗ID"]==s) & (df_train["商品ID"]==i), "売上個数"].sum()
            list.append(m)
            list.append(i)
            list.append(s)
            list.append(sum)
            list_months_shop_id_sum.append(list)

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [135]:
# 月・店舗・商品ごと売上個数で新しいデータフレームを作成

df_train_new = pd.DataFrame(list_months_shop_id_sum,
                            columns=["Months", "商品ID", "店舗ID", "売上個数"]
                            )

In [136]:
# インデックスをリセット
df_train_new = df_train_new.reset_index(drop=True)

In [137]:
# csvに出力

df_train_new.to_csv("train_permonth.csv")

In [138]:
# テストデータに"Month"列を追加

t1 = dt.date(year=2018, month=1, day=1)
t2 = dt.date(year=2019, month=12, day=1)

month = abs(t2.year - t1.year)*12 + (t2.month - t1.month)

df_test.insert(0, "Months", month)

In [139]:
df_test

,Months,商品ID,店舗ID
index,,,
0,23,1000001,0
1,23,1000001,1
2,23,1000001,2
3,23,1000001,3
4,23,1000001,4
...,...,...,...
3055,23,3500001,13
3056,23,3500001,14
3057,23,3500001,15


In [140]:
df_test.to_csv("test_permonth.csv")